## Working with Snapshot camera

Idea: find only the large cluster to reduce detection noise. Then, use the lifespan of customer's cluster to find the most probable vehicle in the snapshot.
Snapshot cam: takes photo every 1:15 second

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster
from countpassenger import Utils
from countpassenger import Snapshot

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

start = 1714284000
stop= 1714288000


pd.set_option("display.precision", 10)
CURRENT_CAM = "mbk-14-11"
SAVE_DIR = os.path.join(conf.RESOURCES_PROCESSED_DIR, f'2024-04-29/count-passengers-{CURRENT_CAM}-vehicle-snapshot.csv')

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-29/mbk-tourist-vehicle-object-20240429.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-29/mbk-tourist-raw-cross-object-20240429.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-29/mbk-tourist-raw-reverse-object-20240429.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle-snapshot")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)

print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT]].to_string())

                          camera       timestamp_precise  timestamp_unix  timestamp_unix_end vehicle_type    xmid    ymid plate_number
1577  mbk-14-11-vehicle-snapshot 2024-04-29 02:00:37.635      1714356037          1714356073        other  1769.5   213.0          NaN
1578  mbk-14-11-vehicle-snapshot 2024-04-29 02:01:52.615      1714356112          1714356149          bus  1760.5   208.0          NaN
1579  mbk-14-11-vehicle-snapshot 2024-04-29 02:03:07.593      1714356187          1714356224        truck  1763.5   209.5       320790
1580  mbk-14-11-vehicle-snapshot 2024-04-29 02:04:22.652      1714356262          1714356299          bus  1760.5   210.5       320790
1561  mbk-14-11-vehicle-snapshot 2024-04-29 02:06:15.104      1714356375          1714356412          NaN  1615.0   272.5          NaN
1562  mbk-14-11-vehicle-snapshot 2024-04-29 02:07:30.083      1714356450          1714356487          NaN  1211.0   354.5          NaN
1538  mbk-14-11-vehicle-snapshot 2024-04-29 02:29:58.80

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

         camera       timestamp_precise    xmid    ymid  timestamp_unix
2289  mbk-14-11 2024-04-29 02:03:45.182  1831.5   394.0      1714356225
2291  mbk-14-11 2024-04-29 02:03:47.662  1290.5   704.5      1714356227
2290  mbk-14-11 2024-04-29 02:03:49.301  1806.0   403.5      1714356229
2262  mbk-14-11 2024-04-29 02:29:07.936  2026.0   278.0      1714357747
2263  mbk-14-11 2024-04-29 02:29:24.451  2079.0   285.5      1714357764
2264  mbk-14-11 2024-04-29 02:29:25.052  2114.0   160.0      1714357765
2260  mbk-14-11 2024-04-29 02:29:31.769  1988.5   377.5      1714357771
2265  mbk-14-11 2024-04-29 02:29:47.925  2058.0   287.5      1714357787
2238  mbk-14-11 2024-04-29 02:45:54.510  2168.0   220.0      1714358754
2239  mbk-14-11 2024-04-29 02:48:51.941  2098.5   223.5      1714358931
2235  mbk-14-11 2024-04-29 02:50:26.998  2064.0   251.0      1714359026
2217  mbk-14-11 2024-04-29 02:54:58.682  2122.5   135.0      1714359298
2218  mbk-14-11 2024-04-29 02:55:02.885  2113.0   135.5      171

### Cluster only large vehicle 

In [3]:
params = dict(
    min_cluster_size=2, #have at least 5 people together
    min_samples=2,
    cluster_selection_epsilon=0, 
    max_cluster_size=30,
    alpha=1.0,
    algorithm="brute",
    n_jobs=None,
    cluster_selection_method="eom", # eom with minsample = 3 is kinda good
    store_centers=None,
)

df_cross, cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross,params=params)
print(Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST][(cluster_cross["timestamp_unix_max"] >= start) & (cluster_cross["timestamp_unix_min"] <= stop)].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1


Empty DataFrame
Columns: [timestamp_min, timestamp_max, count, cluster_id, xmid_mean, ymid_mean]
Index: []


/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/tmp/ipykernel_14128/2290256830.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST][(cluster_cross["timestamp_unix_max"] >= start) & (cluster_cross["timestamp_unix_min"] <= stop)].to_string())


In [4]:
params = dict(
    min_cluster_size=2,
    min_samples=None,
    cluster_selection_epsilon=0,
    max_cluster_size=30,
    alpha=1.0,
    algorithm="brute",
    cluster_selection_method="eom",
    store_centers=None,
)

df_reverse, cluster_reverse = Cluster.perform_reverse_clustering(df_reverse=df_reverse,params=params)
print(Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[(cluster_reverse["timestamp_unix_max"] >= start) & (cluster_reverse["timestamp_unix_min"] <= stop)][conf.CLUSTER_INTEREST].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))


Empty DataFrame
Columns: [timestamp_min, timestamp_max, count, cluster_id, xmid_mean, ymid_mean]
Index: []


/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/tmp/ipykernel_14128/3749486777.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[(cluster_reverse["timestamp_unix_max"] >= start) & (cluster_reverse["timestamp_unix_min"] <= stop)][conf.CLUSTER_INTEREST].to_string())


lifespan of customer's cluster to find the most probable vehicle in the snapshot.
Snapshot cam: takes photo every 1:15 second

nearest x,y ? in time windows?

In [5]:
ken = Snapshot.match_cross_cluster_snapshot_to_vehicle(df_vehicle_snapshot=df_vehicle, cluster_cross=cluster_cross,capture_window=70)
johnny = Snapshot.match_reverse_cluster_snapshot_to_vehicle(df_vehicle_snapshot=ken, cluster_reverse=cluster_reverse,capture_window=70)
johnny[(johnny["timestamp_unix"] >= start) & (johnny["timestamp_unix_end"] <= stop)][conf.VEHICLE_INTEREST_SNAPSHOT_CLUSTERED]

,camera,timestamp_precise,cross_count,reverse_count,vehicle_type,cluster_cross_list,cluster_reverse_list,plate_number,xmid,ymid


Exporting as csv

In [6]:

johnny.to_csv(path_or_buf=SAVE_DIR, index = False)